<a href="https://colab.research.google.com/github/kittimaxz/Study/blob/main/Test_CronJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-analytics-data
!pip install mysql-connector

In [2]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest
import mysql.connector

In [3]:
# สร้างไคลเอนต์เพื่อเชื่อมต่อกับ GA4
client = BetaAnalyticsDataClient()
client

In [4]:
# กำหนดการร้องขอข้อมูล
request = RunReportRequest(
    property="307754729",  # ใส่ Property ID ของ GA4
    dimensions=[Dimension(name="city"), Dimension(name="country")],
    metrics=[Metric(name="activeUsers")],
    date_ranges=[DateRange(start_date="2023-01-01", end_date="today")]
)
request

property: "307754729"
dimensions {
  name: "city"
}
dimensions {
  name: "country"
}
metrics {
  name: "activeUsers"
}
date_ranges {
  start_date: "2023-01-01"
  end_date: "today"
}

In [5]:
# รันการดึงข้อมูล
response = client.run_report(request)
response

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7aa78c585690>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go

ServiceUnavailable: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7aa78c3d5630>)

In [ ]:
# พิมพ์ข้อมูลออกมา
for row in response.rows:
    print(row)

In [ ]:
# เชื่อมต่อกับ MySQL
connection = mysql.connector.connect(
    host="159.138.255.180",
    user="root",
    password="Vcb168899@",
    database="vcb_ba2"
)
connection

In [ ]:
cursor = connection.cursor()
cursor

In [ ]:
# สมมติว่ามีการสร้างตารางล่วงหน้าแล้ว
sql = "INSERT INTO test_backup_ga4(country, city, active_users) VALUES (%s, %s, %s)"
sql

In [ ]:
for row in response.rows:
    data = (row.dimension_values[0].value, row.dimension_values[1].value, row.metric_values[0].value)
    cursor.execute(sql, data)

In [ ]:
connection.commit() # save การเปลี่ยนแปลง
connection.close() # ปิดการเชื่อมต่อ